# DisSysLab — Private Colab Wizard (Wheel Upload)

This notebook is for **private testing** of DisSysLab via a **wheel (.whl) upload**.

**Quick steps**
1. ▶ Run **0) Preflight** (fixes a common Colab warning).
2. ▶ Run **1A) Install DisSysLab from an uploaded wheel** and pick your `dist/dsl-<version>-py3-none-any.whl`.
   - Or use **1B) Alternate: Drag-and-drop then pip install** if you prefer the Files pane.
3. ▶ Run **2) Import sanity check**.
4. ▶ Try the **token-free demos** and the **quick-start pipeline**.
5. (Optional) Enable GPT features if you want to test models.


## 0) Preflight (one-time per session)

In [ ]:
from pathlib import Path
import os

p = Path.cwd()
while not (p / "pyproject.toml").exists():
    if p.parent == p:
        raise FileNotFoundError("pyproject.toml not found above this notebook")
    p = p.parent

os.chdir(p)
print("CWD ->", os.getcwd())  # should print .../DisSysLab


## 1A) Install DisSysLab from an **uploaded wheel**

In [ ]:
from google.colab import files
import glob, os, sys

print("Please upload your DisSysLab wheel (.whl) from your local 'dist/' folder…")
uploads = files.upload()  # opens picker
wheel_paths = [p for p in uploads.keys() if p.endswith('.whl')]
if not wheel_paths:
    raise RuntimeError("No .whl file uploaded. Re-run and choose a file like 'dsl-0.1.0-py3-none-any.whl'.")

wheel = wheel_paths[0]
print("Installing:", wheel)
!pip install -q --upgrade pip
!pip install -q --no-cache-dir "{wheel}"

print("\nInstalled wheels in this directory:")
print("\n".join(glob.glob("*.whl")))
print("\nIf you rebuild locally later, upload the new wheel and run this cell again (use --force-reinstall if needed).")

## 1B) Alternate: Drag-and-drop then pip install

Drag your `.whl` into the **Files** pane (left sidebar). It will land under `/content`. Then run:

In [ ]:
# Example (edit the filename to match yours):
# !pip install -q --upgrade pip
# !pip install -q --no-cache-dir /content/dsl-0.1.0-py3-none-any.whl
print("Edit the cell with your actual wheel filename if you use drag-and-drop.")

## 1C) Verify installed package version (optional)

In [ ]:
import importlib.metadata as m
try:
    print("Installed 'dsl' distribution version:", m.version("dsl"))
except m.PackageNotFoundError:
    print("Could not find installed 'dsl' distribution. Did the pip install succeed?")

## 2) Import sanity check

In [ ]:
try:
    from dsl.core import Network
    from dsl.block_lib.stream_generators import generate
    from dsl.block_lib.stream_transformers import WrapFunction
    from dsl.block_lib.stream_recorders import RecordToList
    print("✅ DisSysLab imports succeeded.")
except Exception as e:
    print("❌ Import failed. Check your wheel build and package layout.")
    raise

## 3) Token-free demos (no API key)

### 3a) String transforms

In [ ]:
def to_uppercase(s: str) -> str:
    return s.upper()

def reverse_text(s: str) -> str:
    return s[::-1]

def word_count(s: str) -> int:
    return len([w for w in s.split() if w.strip()])

samples = [
    "Hello DisSysLab",
    "Agents collaborate by exchanging messages",
    "This runs without any API key"
]

print("Uppercase:", [to_uppercase(x) for x in samples])
print("Reversed:", [reverse_text(x) for x in samples])
print("Word counts:", [word_count(x) for x in samples])

### 3b) Simple numeric scaling

In [ ]:
def zscore(xs):
    if not xs:
        return []
    n = len(xs)
    mean = sum(xs)/n
    var = sum((x-mean)**2 for x in xs)/n
    std = var**0.5 if var>0 else 1.0
    return [(x-mean)/std for x in xs]

xs = [1, 2, 3, 4, 5]
print("Input:", xs)
print("Z-scored:", zscore(xs))

## 4) DisSysLab quick-start (token-free)

In [ ]:
results = []

from dsl.core import Network
from dsl.block_lib.stream_generators import generate
from dsl.block_lib.stream_transformers import WrapFunction
from dsl.block_lib.stream_recorders import RecordToList

def to_upper(s: str) -> str:
    return s.upper()

net = Network(
    blocks={
        "gen": generate(["hello", "distributed", "systems"], key=None),
        "xf": WrapFunction(transform_fn=to_upper),
        "rec": RecordToList(results),
    },
    connections=[("gen", "out", "xf", "in"), ("xf", "out", "rec", "in")],
)

net.compile_and_run()
print("Results:", results)

## 5) (Optional) Enable GPT features

In [ ]:
# Only run if you want to test GPT-powered blocks today.
!pip install -q --upgrade openai

import os
from getpass import getpass

if not os.environ.get("OPENAI_API_KEY"):
    key = getpass("Enter your OpenAI API key (input hidden): ").strip()
    if key:
        os.environ["OPENAI_API_KEY"] = key
        print("✅ OPENAI_API_KEY set for this session.")
    else:
        print("No key provided. You can re-run this cell later.")

In [ ]:
# Minimal GPT smoke test (optional)
try:
    from openai import OpenAI
    client = OpenAI()
    model = input("Enter a chat model you have access to (e.g., 'gpt-4o-mini'): ").strip()
    if model:
        resp = client.chat.completions.create(
            model=model,
            messages=[
                {"role":"system","content":"You are a concise assistant."},
                {"role":"user","content":"Reply with 'ready' only."}
            ],
            max_tokens=5,
            temperature=0
        )
        print("Model responded:", resp.choices[0].message.content)
    else:
        print("Skipping GPT test — no model provided.")
except Exception as e:
    print("GPT test failed:", repr(e))
    print("Tips: check your model name & access; ensure OPENAI_API_KEY is set.")

## 6) Troubleshooting

- **Import failed**: Rebuild the wheel and confirm `packages.find` includes `dsl*`. Verify your wheel file name and that `pip install` succeeded.
- **Model not found**: Use a model your account has access to. Set `OPENAI_API_KEY` in the prior cell.
- **Colab reset**: If the notebook idles, re-run from the top and re-upload the wheel.
- **Force reinstall** after uploading a new wheel:
  ```python
  !pip install -q --no-cache-dir --force-reinstall /content/dsl-<ver>-py3-none-any.whl
  ```
